In [1]:
# OUTDATED: not SOTA approach

# Evaluation of AL finetuning models

# 1. Slice train dataset into the non-labeled part
# 2. Evaluation loop based on non-labeled part 

In [ ]:
# Cloud install
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install seqeval

In [2]:
# import
import pandas as pd
import numpy as np
from datasets import load_dataset
from evaluate import evaluator
from transformers import pipeline

In [24]:
# slice data based on incremental al_step increase
# model trained on 10% is validated on remaining 90% unlabeled train data

data = load_dataset("conll2003")

sliced_data = {}

for i in range(9): 
    sliced_data[i] = data['train'].select(range((i+1)*1000+1, 10000)).shuffle()

Found cached dataset conll2003 (C:/Users/dekai/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
models = [
    'dxiao/bert-finetuned-ner-10percent',
    'dxiao/bert-finetuned-ner-20percent',
    'dxiao/bert-finetuned-ner-30percent',
    'dxiao/bert-finetuned-ner-40percent',
    'dxiao/bert-finetuned-ner-50percent',
    'dxiao/bert-finetuned-ner-60percent',
    'dxiao/bert-finetuned-ner-70percent',
    'dxiao/bert-finetuned-ner-80percent',
    'dxiao/bert-finetuned-ner-90percent' #no 100percent model, assume gold standard set
]

task_evaluator = evaluator("token-classification")

results = []
al_step = 0
for model in models:
    print(al_step)
    print(sliced_data[al_step].num_rows)
    print(model)
    results.append(
        task_evaluator.compute(
            model_or_pipeline=model, data=sliced_data[al_step], metric="seqeval"
            )
        )
    al_step += 1
    

df = pd.DataFrame(results, index=models)
df[["overall_f1", "overall_accuracy", "total_time_in_seconds", "samples_per_second", "latency_in_seconds"]]

0
8999
dxiao/bert-finetuned-ner-10percent


  0%|          | 0/8999 [00:00<?, ?ex/s]

1
7999
dxiao/bert-finetuned-ner-20percent


  0%|          | 0/7999 [00:00<?, ?ex/s]

2
6999
dxiao/bert-finetuned-ner-30percent


  0%|          | 0/6999 [00:00<?, ?ex/s]

KeyboardInterrupt: 